### Project: Data Lake for E-commerce Data Analytics Using Real-Life Dataset

##### Setup environment

In [1]:
!pip install pyspark pandas kaggle


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 KB 1.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 KB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 KB 4.4 MB/s eta 0:00:00
Using legacy 'setup.py install' for kaggle, since package 'wheel' is not installed.
  Running setup.py install for kaggle ... done


In [3]:
!kaggle datasets download -d olistbr/brazilian-ecommerce -p ./brazil_data_lake/raw --unzip


Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
100%|██████████████████████████████████████| 42.6M/42.6M [00:04<00:00, 9.97MB/s]
100%|██████████████████████████████████████| 42.6M/42.6M [00:04<00:00, 9.81MB/s]


#### Ingest the data into the datalake

In [5]:
from pathlib import Path

# Define directories
base_dir = Path('./brazil_data_lake')
raw_dir = base_dir / 'raw'
processed_dir = base_dir / 'processed'
cleaned_dir = base_dir / 'cleaned'

# Create directories
for dir in [raw_dir, processed_dir, cleaned_dir]:
    dir.mkdir(parents=True, exist_ok=True)


#### Process Data (ETL)

In [6]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName('EcommerceDataLake').getOrCreate()

# Load data into Spark DataFrames
orders_df = spark.read.csv(str(raw_dir / 'olist_orders_dataset.csv'), header=True, inferSchema=True)
customers_df = spark.read.csv(str(raw_dir / 'olist_customers_dataset.csv'), header=True, inferSchema=True)
order_items_df = spark.read.csv(str(raw_dir / 'olist_order_items_dataset.csv'), header=True, inferSchema=True)
products_df = spark.read.csv(str(raw_dir / 'olist_products_dataset.csv'), header=True, inferSchema=True)

# Example Transformation: Join orders with customer data and order items
enriched_orders_df = orders_df \
    .join(customers_df, on="customer_id", how="left") \
    .join(order_items_df, on="order_id", how="left") \
    .join(products_df, on="product_id", how="left")

# Save processed data
enriched_orders_df.write.parquet(str(processed_dir / 'orders_enriched.parquet'))


your 131072x1 screen size is bogus. expect trouble
24/08/25 08:32:08 WARN Utils: Your hostname, DOOM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/25 08:32:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/25 08:32:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/25 08:32:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/08/25 08:32:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/08/25 08:32:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memo